<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_Long-Short-Term-Memory/blob/main/LSTM_tf2_0_verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)

#length = 1800

Start = 2700
End = 3600
#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GAN/pytorch/Timeseries-GAN/proof/train_veri.txt')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GAN/pytorch/Timeseries-GAN/proof/test_veri.txt')
train_data = df_train.iloc[Start : End,:].values
test_data = df_test.iloc[Start : End,:].values
print(train_data.shape)
print(train_data)


trainSet = train_data
testSet = test_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization

#print("Before Norm : ",train_data_input.shape, train_data_input)
#soc_max = 95
#soc_min = 70

def origin_minmax(data) : 
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])
  print(arr_min)
  print(data.shape[1])
  for i in range(data.shape[1]) : 
    #print(i)
    max = np.max(data[:,i])
    min = np.min(data[:,i]) 
    #if i == 2 : 
      #max = soc_max
      #min = soc_min
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(trainSet)
arr_max2, arr_min2 = origin_minmax(testSet)
print(arr_max1, arr_min1)

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(trainSet[:,i])
    min = np.min(trainSet[:,i])
    #if i == 2 : 
      #max = soc_max
      #min = soc_min
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data
print(testSet.shape)
testSet = MinMaxScaler(testSet)
trainSet = MinMaxScaler(trainSet)

def back_MinMax(data,max,min):
  #print(max)
  #print(min)
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (max[i] - min[i]) + min[i]
    pass
  return data

print("trainSet : ",arr_max1, arr_min1)
print("testSet : ",arr_max2, arr_min2)
print("trainSet : ",trainSet)
print("testSet : ",testSet)

In [ ]:
#Hyper parameters
seqLength = 10
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 50
lr = 0.001
iterations = 30
batch_size = 32
'''
trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)
'''

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, time_series.shape[0] - seq_length): 
        #print(i)
        _x = time_series[i:i + seq_length, : -1 ]
        _y = time_series[i + seq_length -1, [-1]]  
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet, seqLength)
testX, testY = sliding_window(testSet, seqLength)

print(trainX.shape)
print(trainY.shape)



In [ ]:
# Stacked LSTM Modeling
# Return Sequences is very important for stacking
# This means weather output is Sequence of this layer or all of sequences
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
input_dim = trainSet.shape[1]-1
model = keras.Sequential()
model.add(layers.LSTM(units = hiddenDim, activation='ReLU',input_shape = (seqLength,input_dim), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'ReLU',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'ReLU',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'ReLU',return_sequences = False, kernel_initializer = initializer))
model.add(layers.Dense(outputDim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()


In [ ]:
# Training
hist = model.fit(trainX, trainY, epochs = iterations, batch_size = batch_size)

In [ ]:
# Evaluate Testset
res = model.evaluate(testX, testY, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = trainX
yhat = model.predict(xhat)
#print(testY)
#sprint(yhat)
#print("Evaluate : {}".format(np.sqrt(np.average((yhat - testY)**2))))

In [ ]:
#Estimating
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GAN/pytorch/Timeseries-GAN/proof/train_veri.txt')
testSet_for_result = df_test.iloc[Start:End,:].values
maxs = np.max(testSet_for_result[:,-1])
mins = np.min(testSet_for_result[:,-1])
print(maxs,mins)
estimated = yhat
estimated = yhat*(maxs - mins) + mins
trainY = trainY * (maxs - mins) + mins
#testSet_for_result = df_test
real = df_test.iloc[Start:End,-1] # estimated + seqLength
print("estimated : ", estimated.shape)
print("real : ", real.shape)

#RMSE
Error1 = np.sqrt(np.sum(np.power(trainY/180- estimated/180,2))/len(estimated))
print("RMSE : ",Error1)


In [ ]:
from pandas._libs.tslibs.conversion import ensure_datetime64ns
plt.figure()
time1 = 0
time2 = 900
plt.plot(estimated,label = "Estimated by Stacked LSTM")
plt.plot(trainY, label = "Real")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(estimated,label = "Estimated by Stacked LSTM")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(trainY, label = "Real")
plt.legend(prop = {'size':10})

print(estimated.shape)
print(real.shape)

In [ ]:
# Excel Data
import pandas as pd

#Change the sheet nam4
from0_900 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/GAN/pytorch/Timeseries-GAN/proof/정리3(2700_3600).xlsx', sheet_name = 24, usecols = [0,1,2,4], nrows = 300, header = None, skiprows = 1)
print(from0_900.columns)
#0,1,2,4 GAN SOC
#from0_900 = from0_900[[0,1,7,4]]
from0_900 = round(from0_900,1)
from0_900 = from0_900.values

#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, time_series.shape[0] - seq_length): 
        #print(i)
        _x = time_series[i:i + seq_length, : -1 ]
        _y = time_series[i + seq_length -1, [-1]]  
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

print(from0_900)
print(from0_900.shape)

In [ ]:
def MinMaxScaler_GAN(data):
  for i in range(data.shape[1]) : 
    max = arr_max1[i]
    min = arr_min1[i]
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

veri_GAN = MinMaxScaler_GAN(from0_900)
print(veri_GAN)

test_gan_x, test_gan_y = sliding_window(from0_900, seqLength)


In [ ]:
xhat_gan = test_gan_x
yhat_gan = model.predict(xhat_gan)

In [ ]:
yhat_gan = yhat_gan * (arr_max1[-1] - arr_min1[-1]) + arr_min1[-1]
test_gan_y = test_gan_y * (arr_max1[-1] - arr_min1[-1]) + arr_min1[-1]
print(arr_max1[-1])
print(arr_min1[-1])
#print(yhat_gan)


In [ ]:
plt.figure()
plt.plot(yhat_gan/180,label = "Reference Voltage")
plt.legend(prop = {'size':10})

plt.plot(test_gan_y/180,label = "Generated Voltage by GAN")
plt.legend(prop = {'size':10})
plt.legend(loc = 'upper right')
plt.grid()
plt.xlabel('Time[sec]')
plt.ylabel('Voltage[V]')
plt.savefig('/content/drive/MyDrive/Colab Notebooks/GAN/pytorch/Timeseries-GAN/proof/220330/2700to3600/GANSOC_25.png')

In [ ]:
#Root Mean Square Error
import numpy as np
print(yhat_gan.shape)
print(test_gan_y.shape)

Error = np.sqrt(np.sum(np.power(yhat_gan/180-test_gan_y/180,2))/len(yhat_gan))
print("Error_GAN : ",Error)
print("Error_DL model : ",Error1)
print("RMSE : ",Error-Error1)

np.savetxt("/content/drive/MyDrive/Colab Notebooks/GAN/pytorch/Timeseries-GAN/proof/220330/yhat_gan", yhat_gan)